# Test

In [1]:
import molsysmt as msm

In [2]:
import openmm as mm
from openmm import app
from openmm import unit

In [3]:
#molsys = msm.convert('4POC')
#molsys = msm.extract(molsys, selection='molecule_type=="protein"')
#molsys = msm.build.add_missing_hydrogens(molsys)
#molsys = msm.build.solvate(molsys)

In [4]:
molsys = msm.convert('initial.msmpk')

In [5]:
topology = msm.convert(molsys, 'openmm.Topology')
coordinates = msm.get(molsys, coordinates=True)
positions = msm.pyunitwizard.convert(coordinates[0], to_form='openmm.unit')

In [6]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')

In [7]:
system = forcefield.createSystem(topology, nonbondedMethod=app.PME,
        nonbondedCutoff=1.2*unit.nanometer, constraints=app.HBonds)

In [8]:
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 2.0*unit.femtoseconds)

In [9]:
platform = mm.Platform.getPlatformByName("CPU")

In [10]:
simulation = app.Simulation(topology, system, integrator, platform)

In [11]:
simulation.context.setPositions(positions)

In [12]:
#simulation.minimizeEnergy()
#msm.convert(simulation, 'initial.msmpk')

In [13]:
#from molsysmt.thirds.openmm.reporters import MolSysMTTrajectoryDictReporter
from molsysmt.thirds.openmm.reporters import MSMH5Reporter
from mdtraj.reporters import HDF5Reporter
from molsysmt.thirds.openmm.reporters import TQDMReporter

In [14]:
tqdm_reporter = TQDMReporter(100, 5000)
simulation.reporters.append(tqdm_reporter)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [15]:
#h5_reporter = HDF5Reporter('test.h5', 100)
#simulation.reporters.append(h5_reporter)

In [16]:
#dict_reporter = MolSysMTTrajectoryDictReporter(10)
#simulation.reporters.append(dict_reporter)

In [17]:
msmh5_reporter = MSMH5Reporter('test.msmh5', 100, steps=5000, selection='all',
            topology=True, time=True, box=True, coordinates=True, velocities=False,
            potentialEnergy=True, kineticEnergy=True, temperature=True,
            includeInitialContext=True, constantReportInterval=True,
            constantStepSize=True, constantBox=True)
simulation.reporters.append(msmh5_reporter)

In [18]:
%%time
simulation.step(5000)

CPU times: user 47min 57s, sys: 18.4 s, total: 48min 15s
Wall time: 2min 57s


In [ ]:
h5_reporter.close()

In [19]:
msmh5_reporter.close()

In [ ]:
traj = dict_reporter.finalize()
msmh5_reporter.close()

In [ ]:
tqdm : 30 it/s
CPU times: user 47min 36s, sys: 18.1 s, total: 47min 54s
Wall time: 2min 45s

tqdm + h5: 30 it/s
CPU times: user 46min 52s, sys: 17.9 s, total: 47min 10s
Wall time: 2min 44s



